이 노트북은 [케라스 창시자에게 배우는 딥러닝 2판](https://tensorflow.blog/kerasdl2/)의 예제 코드를 담고 있습니다.

<table align="left">
    <tr>
        <td>
            <a href="https://colab.research.google.com/github/rickiepark/deep-learning-with-python-2nd/blob/main/chapter11_part03_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
        </td>
    </tr>
</table>

## 트랜스포머 아키텍처

https://ratsgo.github.io/nlpbook/docs/language_model/tr_self_attention/

### 셀프 어텐션 이해하기

- 트랜스포머 구조에서 멀티 헤드 어텐션은 셀프 어텐션(self attention)이라고도 불립니다. 
- 트랜스포머 경쟁력의 원천은 셀프 어텐션에 있다.
- 어텐션(attention)은 시퀀스 입력에 수행하는 기계학습 방법의 일종으로 시퀀스 요소들 가운데 태스크 수행에 중요한 요소에 집중하고 그렇지 않은 요소는 무시해 태스크 수행 성능을 끌어 올리며 기계 번역 과제에 처음 도입되었다.
- 기계 번역에 어텐션을 도입한다면 타깃 언어를 디코딩할 때 소스 언어의 단어 시퀀스 가운데 디코딩에 도움되는 단어들 위주로 취사 선택해서 번역 품질을 끌어 올리게 된다. 즉, 어텐션은 디코딩할 때 소스 시퀀스 가운데 중요한 요소들만 추린다.
- 셀프 어텐션이란, 말 그대로 자기 자신에 수행하는 어텐션 기법으로 입력 시퀀스 가운데 태스크 수행에 의미 있는 요소들 위주로 정보를 추출한다는 것이다.
- 합성곱 신경망과 비교 : CNN은 합성곱 필터(convoltion filter) 라는 특수한 장치를 이용해 시퀀스의 지역적인 특징을 잡아내는 모델이다. 자연어는 기본적으로 시퀀스(단어 혹은 형태소의 나열)이고 특정 단어 기준 주변 문맥이 의미 형성에 중요한 역할을 하므로 CNN이 자연어 처리에 널리 쓰인다.하지만 CNN은 합성곱 필터 크기를 넘어서는 문맥은 읽어내기 어렵다는 단점이 있습니다. 예컨대 필터 크기가 3(3개 단어씩 처리)이라면 4칸 이상 떨어져 있는 단어 사이의 의미는 캐치하기 어렵다.
- 순환 신경망과 비교 : RNN 역시 시퀀스 정보를 압축하는 데 강점이 있는 구조이다. 소스 언어 시퀀스인 어제, 카페, 갔었어, 거기, 사람, 많더라를 인코딩해야 한다고 가정해 보면 RNN은 소스 시퀀스를 차례대로 처리한다. 하지만 RNN은 시퀀스 길이가 길어질 수록 정보 압축에 문제가 발생하며 오래 전에 입력된 단어는 잊어버리거나, 특정 단어 정보를 과도하게 반영해 전체 정보를 왜곡하는 경우가 자주 생긴다.
기계 번역을 할 때 RNN을 사용한다면 인코더가 디코더로 넘기는 정보는 소스 시퀀스의 마지막인 많더라라는 단어의 의미가 많이 반영될 수밖에 없으며 RNN은 입력 정보를 차례대로 처리하고 오래 전에 읽었던 단어는 잊어버리는 경향이 있다.
- 어텐션과 비교 : cafe에 대응하는 소스 언어의 단어는 카페이고 이는 소스 시퀀스의 초반부에 등장한 상황에서 cafe라는 단어를 디코딩해야 할 때 카페를 반드시 참조해야 한다. 어텐션이 없는 단순 RNN을 사용하면 워낙 초반에 입력된 단어라 모델이 잊었을 가능성이 크고, 이 때문에 번역 품질이 낮아질 수 있다. 어텐션은 이러한 문제점을 해결하기 위해 제안되었으며 디코더 쪽 RNN에 어텐션을 추가하는 방식이다. 어텐션은 디코더가 타깃 시퀀스를 생성할 때 소스 시퀀스 전체에서 어떤 요소에 주목해야 할지 알려주므로 카페가 소스 시퀀스 초반에 등장하거나 소스 시퀀스의 길이가 길어지더라도 번역 품질이 떨어지는 것을 막을 수 있다.
- 셀프 어텐션은 자기 자신에 수행하는 어텐션이다.입력 시퀀스가 어제, 카페, 갔었어, 거기, 사람, 많더라일 때 거기라는 단어가 어떤 의미를 가지는지 계산하는 상황에서 잘 학습된 셀프 어텐션 모델이라면 거기에 대응하는 장소는 카페라는 사실을 알아챌 수 있다. 그뿐만 아니라 거기는 갔었어와도 연관이 있음을 확인할 수 있다. 트랜스포머 인코더 블록 내부에서는 이처럼 거기라는 단어를 인코딩할 때 카페, 갔었어라는 단어의 의미를 강조해서 반영한다.
- 셀프 어텐션 수행 대상은 입력 시퀀스 전체이며 개별 단어와 전체 입력 시퀀스를 대상으로 어텐션 계산을 수행해 문맥 전체를 고려하기 때문에 지역적인 문맥만 보는 CNN 대비 강점이 있다. 아울러 모든 경우의 수를 고려(단어들 서로가 서로를 1대 1로 바라보게 함)하기 때문에 시퀀스 길이가 길어지더라도 정보를 잊거나 왜곡할 염려가 없다. 이는 RNN의 단점을 극복한 지점입이다.

어텐션과 셀프 어텐션의 주요 차이
- 어텐션은 소스 시퀀스 전체 단어들(어제, 카페, …, 많더라)과 타깃 시퀀스 단어 하나(cafe) 사이를 연결하는 데 쓰인다. 반면 셀프 어텐션은 입력 시퀀스 전체 단어들(그림15, 그림16) 사이를 연결한다.
- 어텐션은 RNN 구조 위에서 동작하지만 셀프 어텐션은 RNN 없이 동작한다.
타깃 언어의 단어를 1개 생성할 때 어텐션은 1회 수행하지만 셀프어텐션은 인코더, 디코더 블록의 개수만큼 반복 수행한다.


#### 일반화된 셀프 어텐션: 쿼리-키-값 모델

- 셀프 어텐션은 쿼리(query), 키(key), 밸류(value) 세 가지 요소가 서로 영향을 주고 받는 구조이다. 트랜스포머 블록에는 문장 내 각 단어가 벡터(vector) 형태로 입력되는데 여기서 벡터란 숫자의 나열 정도로 이해할 수 있다.
- 각 단어 벡터는 블록 내에서 어떤 계산 과정을 거쳐 쿼리, 키, 밸류 세 가지로 변환된다. 만일 트랜스포머 블록에 입력되는 문장이 여섯 개 단어로 구성돼 있다면 이 블록의 셀프 어텐션 계산 대상은 쿼리 벡터 6개, 키 벡터 6개, 밸류 백터 6개 등 모두 18개가 된다. 셀프 어텐션은 쿼리 단어 각각에 대해 모든 키 단어와 얼마나 유기적인 관계를 맺고 있는지 그 합이 1인 확률값으로 나타낸다. 카페라는 쿼리 단어와 가장 관련이 높은 키 단어는 거기라는 점(0.4)을 확인할 수 있다.
- 셀프 어텐션 모듈은 밸류 벡터들을 가중합(weighted sum)하는 방식으로 계산을 마무리한다. 새롭게 만들어지는 카페 벡터( Z카페 )는 문장에 속한 모든 단어 쌍 사이의 관계가 녹아 있다.
Z카페=0.1×V어제+0.1×V카페+0.1×V갔었어+0.4×V거기+0.2×V사람+0.1×V많더라
- 카페에 대해서만 계산 예를 들었지만 이러한 방식으로 나머지 단어들도 셀프 어텐션을 각각 수행한다. 모드 시퀀스를 대상으로 셀프 어텐션 계산이 끝나면 그 결과를 다음 블록으로 넘깁니다. 이처럼 트랜스포머 모델은 셀프 어텐션을 블록(레이어) 수만큼 반복합니다.

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


셀프 어텐션 동작 원리

https://ratsgo.github.io/nlpbook/docs/language_model/tr_self_attention/

셀프 어텐션 출력 과정 : 쿼리, 키, 밸류 3개 요소 사이의 문맥적 관계성을 추출하는 과정
- 입력(단어 개수, 단어 임베딩 차원 수)
- 쿼리, 키, 밸류 만들기 : 세가지 행렬은 태스크를 가장 잘 수행하는 방향으로 학습 과정에서 업데이트
- 셀프 어텐션 계산
  - 쿼리 벡터들을 한꺼번에 모아서 키 벡터들과 행렬곱을 수행
  - 소프트맥스 확률값 만들기 : 키 벡터의 차원수의 제곱근으로 나눠준 뒤 소프트맥스를 취하는 과정
  - 소프트맥스 확률과 밸류를 가중합하기

셀프 어텐션 계산

쿼리와 키를 행렬곱한 뒤 해당 행렬의 모든 요소값을 키 차원수의 제곱근 값으로 나눠주고, 이 행렬을 행(row) 단위로 소프트맥스(softmax)*를 취해 스코어 행렬을 만들어줍니다. 이 스코어 행렬에 밸류를 행렬곱해 줘서 셀프 어텐션 계산

In [ ]:
# 변수 정의
import torch

# 입력 벡타 시퀀스(단어 개수 3, 차원 수 4)
x = torch.tensor([
  [1.0, 0.0, 1.0, 0.0],
  [0.0, 2.0, 0.0, 2.0],
  [1.0, 1.0, 1.0, 1.0],  
])

# 셀프 어텐션은 쿼리, 키, 밸류 3개 요소 사이의 문맥적 관계성을 추출하는 과정
# 쿼리, 키, 밸류를 만들어 주는 행렬(w)
w_query = torch.tensor([
  [1.0, 0.0, 1.0],
  [1.0, 0.0, 0.0],
  [0.0, 0.0, 1.0],
  [0.0, 1.0, 1.0]
])
w_key = torch.tensor([
  [0.0, 0.0, 1.0],
  [1.0, 1.0, 0.0],
  [0.0, 1.0, 0.0],
  [1.0, 1.0, 0.0]
])
w_value = torch.tensor([
  [0.0, 2.0, 0.0],
  [0.0, 3.0, 0.0],
  [1.0, 0.0, 3.0],
  [1.0, 1.0, 0.0]
])

In [ ]:
# 쿼리, 키, 밸류 만들기
# w_key,w_query,w_value 세가지 행렬은 태스크를 가장 잘 수행할 수 있는 방향으로 학습 과정에서 업데이트
keys = torch.matmul(x, w_key)
querys = torch.matmul(x, w_query)
values = torch.matmul(x, w_value)
print(keys,'\n')
print(querys,'\n')
print(values)

tensor([[0., 1., 1.],
        [4., 4., 0.],
        [2., 3., 1.]]) 

tensor([[1., 0., 2.],
        [2., 2., 2.],
        [2., 1., 3.]]) 

tensor([[1., 2., 3.],
        [2., 8., 0.],
        [2., 6., 3.]])


In [ ]:
#  쿼리 벡터들을 한꺼번에 모아서 키 벡터들과 행렬곱을 수행
attn_scores = torch.matmul(querys, keys.T)
print(querys,'\n')
print(keys.T,'\n')
attn_scores

tensor([[1., 0., 2.],
        [2., 2., 2.],
        [2., 1., 3.]]) 

tensor([[0., 4., 2.],
        [1., 4., 3.],
        [1., 0., 1.]]) 



tensor([[ 2.,  4.,  4.],
        [ 4., 16., 12.],
        [ 4., 12., 10.]])

In [ ]:
import numpy as np
print(keys,'\n')
print(keys.shape[-1],'\n')
np.sqrt(keys.shape[-1])

tensor([[0., 1., 1.],
        [4., 4., 0.],
        [2., 3., 1.]]) 

3 



1.7320508075688772

In [ ]:
# 소프트맥스 확률값 만들기 : 키 벡터의 차원수의 제곱근으로 나눠준 뒤 소프트맥스를 취하는 과정
import numpy as np
from torch.nn.functional import softmax
key_dim_sqrt = np.sqrt(keys.shape[-1])
attn_probs = softmax(attn_scores / key_dim_sqrt, dim=-1)
attn_probs

tensor([[1.3613e-01, 4.3194e-01, 4.3194e-01],
        [8.9045e-04, 9.0884e-01, 9.0267e-02],
        [7.4449e-03, 7.5471e-01, 2.3785e-01]])

In [ ]:
# 소프트맥스 확률과 밸류를 가중합하기
weighted_values = torch.matmul(attn_probs, values)
print(values,'\n')
weighted_values

tensor([[1., 2., 3.],
        [2., 8., 0.],
        [2., 6., 3.]]) 



tensor([[1.8639, 6.3194, 1.7042],
        [1.9991, 7.8141, 0.2735],
        [1.9926, 7.4796, 0.7359]])

- 셀프 어텐션의 학습 대상은 쿼리, 키, 밸류를 만드는 가중치 행렬입니다. 
- 코드 예시에서는 w_query, w_key, w_value입니다. 
- 이들은 태스크(예: 기계 번역)를 가장 잘 수행하는 방향으로 학습 과정에서 업데이트

### 멀티 헤드 어텐션

- 셀프 어텐션(self attention)을 여러 번 수행한 걸 가리킵니다. 
- 여러 헤드가 독자적으로 셀프 어텐션을 계산한다는 이야기입니다. 비유하자면 같은 문서(입력)를 두고 독자(헤드) 여러 명이 함께 읽는 구조
- 멀티-헤드 어텐션은 개별 헤드의 셀프 어텐션 수행 결과를 이어붙인 행렬에  WO 를 행렬곱해서 마무리
- 멀티-헤드 어텐션의 최종 수행 결과는 ‘입력 단어 수  ×  목표 차원수’
- 멀티 헤드 어텐션은 인코더, 디코더 블록 모두에 적용됩니다. 앞으로 특별한 언급이 없다면 셀프 어텐션은 멀티 헤드 어텐션인 것으로 이해

트랜스포머 모델 학습 및 인퍼런스 : 기계 번역을 수행(인퍼런스)
 - 소스 언어(한국어) 문장을 인코더에 입력해 인코더 마지막 블록의 단어 벡터 시퀀스를 추출
 - 인코더에서 넘어온 소스 언어 문장 정보와 디코더에 타깃 문장 시작을 알리는 스페셜 토큰 $<s>$를 넣어서, 타깃 언어(영어)의 첫 번째 토큰을 생성
 - 인코더 쪽에서 넘어온 소스 언어 문장 정보와 이전에 생성된 타깃 언어 토큰 시퀀스를 디코더에 넣어서 만든 정보로 타깃 언어의 다음 토큰을 생성
 - 생성된 문장 길이가 충분하거나 문장 끝을 알리는 스페셜 토큰 $</s>$가 나올 때까지 3을 반복

트랜스포머에 적용된 기술
- 인코더와 디코더 블록의 구조는 멀티 헤드 어텐션, 피드포워드 뉴럴 네트워크, 잔차 연결 및 레이어 정규화 등 세 가지 구성 요소를 기본
- 피드포워드 뉴럴네트워크
  - 입력은 현재 블록의 멀티 헤드 어텐션의의 개별 출력 벡터
  - 피드포워드 뉴럴네트워크란 신경망(neural network)의 한 종류로 그림2와 같이 입력층(input layer,  x ), 은닉층(hidden layer,  h ), 출력층(ouput layer,  y ) 3개 계층으로 구성
  - 이전 뉴런 값과 그에 해당하는 가중치를 가중합(weighted sum)한 결과에 바이어스(bias)를 더해 만듭니다. 가중치들과 바이어스는 학습 과정에서 업데이트. 활성 함수(activation function,  f )는 현재 계산하고 있는 뉴런의 출력을 일정 범위로 제한하는 역할. 활성함수는 ReLU(Rectified Linear Unit)
  - 입력층 뉴런이 각각  [2,1] 이고 그에 해당하는 가중치가  [3,2] , 바이어스(bias)가 1이라고 가정해 보겠습니다. 그러면 은닉층 첫번째 뉴런 값은  2×3+1×2+1=9 가 되며 이 값은 양수이므로 ReLU를 통과해도 그대로 유지
  - 트랜스포머에서는 은닉층의 뉴런 갯수(즉 은닉층의 차원수)를 입력층의 네 배로 설정. 예컨대 피드포워드 뉴럴네트워크의 입력 벡터가 768차원일 경우 은닉층은 2048차원까지 늘렸다가 출력층에서 이를 다시 768차원으로 줄인다.

- 잔차 연결
  - 잔차 연결이란 블록(block) 계산을 건너뛰는 경로를 하나 두는 것을 의미
  - 입력을  x , 이번 계산 대상 블록을  F 라고 할 때 잔차 연결은  F(x)+x 로 간단히 실현
  - 잔차 연결을 두지 않았을 때는  f1 ,  f2 ,  f3 을 연속으로 수행하는 경로 한 가지만 존재하였으나, 잔차 연결을 블록마다 설정해둠으로써 모두 8가지의 새로운 경로가 생성. 다시 말해 모델이 다양한 관점에서 블록 계산을 수행
  - 잔차 연결은 모델 중간에 블록을 건너뛰는 경로를 설정함으로써 학습을 용이하게 하는 효과가 있음

- 레이어 정규화
  - 미니 배치의 인스턴스( x )별로 평균을 빼주고 표준편차로 나눠줘 정규화(normalization)을 수행하는 기법
  - 레이어 정규화를 수행하면 학습이 안정되고 그 속도가 빨라지는 등의 효과

- 드롭아웃
  - 딥러닝 모델은 그 표현력이 아주 좋아서 학습 데이터 그 자체를 외워버릴 염려가 있습니다. 이를 과적합(overfitting)이라고 합니다. 드롭아웃(dropout)은 이러한 과적합 현상을 방지하고자 뉴런의 일부를 확률적으로 0으로 대치하여 계산에서 제외하는 기법
  - torch.nn.Dropout 객체는 뉴런별로 드롭아웃을 수행할지 말지를 확률적으로 결정하는 함수인데 p=0.2라는 말은 드롭아웃 수행 비율이 평균적으로 20%가 되게끔 하는 것임

- 옵티마이저
  - 딥러닝 모델 학습은 모델 출력과 정답 사이의 오차(error)를 최소화하는 방향을 구하고 이 방향에 맞춰 모델 전체의 파라미터(parameter)들을 업데이트하는 과정. 이때 오차를 최소화하는 방향을 그래디언트(gradient)라고 하며 오차를 최소화하는 과정을 최적화(optimization)라고 함
  - 오차를 구하려면 현재 시점의 모델에 입력을 넣어봐서 처음부터 끝까지 계산해보고 정답과 비교해야 하며 오차를 구하기 위해 이같이 모델 처음부터 끝까지 순서대로 계산해보는 과정을 순전파(forward propagation)이라고 한다.
  - 오차를 구했다면 오차를 최소화하는 최초의 그래디언트를 구할 수 있으며 이는 미분(devative)으로 구합니다. 이후 미분의 연쇄 법칙(chain rule)에 따라 모델 각 가중치별 그래디언트 역시 구할 수 있다. 이 과정은 순전파의 역순으로 순차적으로 수행되는데 이를 역전파(backpropagation)라고 함
  - 학습 과정은 미니 배치 단위로 이루어지며 이는 눈을 가린 상태에서 산등성이를 한걸음씩 내려가는 과정에 비유할 수 있다. 내가 지금 있는 위치에서 360도 모든 방향에 대해 한발한발 내딛어보고 가장 경사가 급한 쪽으로 한걸음씩 내려가는 과정을 반복하는 것이다.
  - 모델을 업데이트할 때(산등성이를 내려갈 때) 중요한 것은 방향과 보폭인데 이는 최적화 도구(optimizer)의 도움을 받는다. 트랜스포머 모델이 쓰는 최적화 도구가 바로 아담 옵티마이저(Adam Optimizer)이며 아담 옵티마이저는 오차를 줄이는 성능이 좋아서 트랜스포머 말고도 널리 쓰인다.
  - 아담 옵티마이저의 핵심 동작 원리는 방향과 보폭을 적절하게 정해주는 것이다. 방향을 정할 때는 현재 위치에서 가장 경사가 급한 쪽으로 내려가되, 여태까지 내려오던 관성(방향)을 일부 유지하도록 한다. 보폭의 경우 안가본 곳은 성큼 빠르게 걸어 훑고 많이 가본 곳은 갈수록 보폭을 줄여 세밀하게 탐색하는 방식으로 정한다.

In [ ]:
import torch
x = torch.tensor([2,1])
w1 = torch.tensor([[3,2,-4],[2,-3,1]])
b1 = 1
w2 = torch.tensor([[-1, 1], [1,2], [3,1]])
b2 = -1

In [ ]:
h_preact = torch.matmul(x, w1) + b1
h = torch.nn.functional.relu(h_preact)
y = torch.matmul(h, w2) + b2
print(h_preact,'\n')
print(h,'\n')
y

tensor([ 9,  2, -6]) 

tensor([9, 2, 0]) 



tensor([-8, 12])

In [ ]:
# 레이어 정규화 예시
import torch
input = torch.tensor([[1.0, 2.0, 3.0], [1.0, 1.0, 1.0]])
m = torch.nn.LayerNorm(input.shape[-1])
output = m(input)
output

tensor([[-1.2247,  0.0000,  1.2247],
        [ 0.0000,  0.0000,  0.0000]], grad_fn=<NativeLayerNormBackward0>)

In [ ]:
print(m.weight,'\n')
print(m.bias)

Parameter containing:
tensor([1., 1., 1.], requires_grad=True) 

Parameter containing:
tensor([0., 0., 0.], requires_grad=True)


### 트랜스포머 인코더

**데이터 가져오기**

In [ ]:
!rm -r aclImdb
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!rm -r aclImdb/train/unsup

rm: cannot remove 'aclImdb': No such file or directory
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  2824k      0  0:00:29  0:00:29 --:--:-- 1974k


**데이터 준비**

In [ ]:
import os, pathlib, shutil, random
from tensorflow import keras
batch_size = 32
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"
for category in ("neg", "pos"):
    os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)
    num_val_samples = int(0.2 * len(files))
    val_files = files[-num_val_samples:]
    for fname in val_files:
        shutil.move(train_dir / category / fname,
                    val_dir / category / fname)

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)
text_only_train_ds = train_ds.map(lambda x, y: x)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


**데이터 벡터화**

In [ ]:
from tensorflow.keras import layers

max_length = 600
max_tokens = 20000
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)
text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

**`Layer` 층을 상속하여 구현한 트랜스포머 인코더**

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

**트랜스포머 인코더를 사용하여 텍스트 분류하기**

In [ ]:
vocab_size = 20000
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")
x = layers.Embedding(vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 256)         5120000   
                                                                 
 transformer_encoder (Transf  (None, None, 256)        543776    
 ormerEncoder)                                                   
                                                                 
 global_max_pooling1d (Globa  (None, 256)              0         
 lMaxPooling1D)                                                  
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 257   

**트랜스포머 인코더 기반 모델 훈련하고 평가하기**

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint("transformer_encoder.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=20, callbacks=callbacks)
model = keras.models.load_model(
    "transformer_encoder.keras",
    custom_objects={"TransformerEncoder": TransformerEncoder})
print(f"테스트 정확도: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/20
625/625 [==============================] - 45s 63ms/step - loss: 0.4759 - accuracy: 0.7742 - val_loss: 0.3358 - val_accuracy: 0.8582
Epoch 2/20
625/625 [==============================] - 40s 64ms/step - loss: 0.3128 - accuracy: 0.8675 - val_loss: 0.2905 - val_accuracy: 0.8796
Epoch 3/20
625/625 [==============================] - 41s 65ms/step - loss: 0.2428 - accuracy: 0.9009 - val_loss: 0.2768 - val_accuracy: 0.8874
Epoch 4/20
625/625 [==============================] - 42s 66ms/step - loss: 0.1935 - accuracy: 0.9252 - val_loss: 0.2777 - val_accuracy: 0.8906
Epoch 5/20
625/625 [==============================] - 42s 66ms/step - loss: 0.1538 - accuracy: 0.9416 - val_loss: 0.2862 - val_accuracy: 0.8948
Epoch 6/20
625/625 [==============================] - 41s 65ms/step - loss: 0.1289 - accuracy: 0.9523 - val_loss: 0.3030 - val_accuracy: 0.8852
Epoch 7/20
625/625 [==============================] - 41s 66ms/step - loss: 0.1093 - accuracy: 0.9589 - val_loss: 0.3418 - val_accuracy:

#### 위치 인코딩을 사용해 위치 정보 주입하기

**서브클래싱으로 위치 임베딩 구현하기**

In [ ]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

#### 텍스트 분류 트랜스포머

**트랜스포머 인코더와 위치 임베딩 합치기**

In [ ]:
vocab_size = 20000
sequence_length = 600
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("full_transformer_encoder.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=20, callbacks=callbacks)
model = keras.models.load_model(
    "full_transformer_encoder.keras",
    custom_objects={"TransformerEncoder": TransformerEncoder,
                    "PositionalEmbedding": PositionalEmbedding})
print(f"테스트 정확도: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 positional_embedding (Posit  (None, None, 256)        5273600   
 ionalEmbedding)                                                 
                                                                 
 transformer_encoder_1 (Tran  (None, None, 256)        543776    
 sformerEncoder)                                                 
                                                                 
 global_max_pooling1d_1 (Glo  (None, 256)              0         
 balMaxPooling1D)                                                
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                           

### BoW 모델 대신 언제 시퀀스 모델을 사용하나요?